In [1]:
!wget https://raw.githubusercontent.com/mrdbourke/tensorflow-deep-learning/main/extras/helper_functions.py

--2024-05-16 07:57:09--  https://raw.githubusercontent.com/mrdbourke/tensorflow-deep-learning/main/extras/helper_functions.py
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.110.133, 185.199.109.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 10246 (10K) [text/plain]
Saving to: ‘helper_functions.py’

helper_functions.py 100%[===================>]  10.01K  --.-KB/s    in 0s      

2024-05-16 07:57:09 (41.1 MB/s) - ‘helper_functions.py’ saved [10246/10246]



In [2]:
from helper_functions import unzip_data, create_tensorboard_callback, plot_loss_curves, compare_historys

In [3]:
!wget https://storage.googleapis.com/ztm_tf_course/nlp_getting_started.zip

--2024-05-16 07:57:19--  https://storage.googleapis.com/ztm_tf_course/nlp_getting_started.zip
Resolving storage.googleapis.com (storage.googleapis.com)... 74.125.134.207, 74.125.139.207, 74.125.141.207, ...
Connecting to storage.googleapis.com (storage.googleapis.com)|74.125.134.207|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 607343 (593K) [application/zip]
Saving to: ‘nlp_getting_started.zip’

nlp_getting_started 100%[===================>] 593.11K  --.-KB/s    in 0.008s  

2024-05-16 07:57:19 (74.5 MB/s) - ‘nlp_getting_started.zip’ saved [607343/607343]



In [4]:
unzip_data("nlp_getting_started.zip")

In [5]:
import pandas as pd

train_df = pd.read_csv("train.csv")
test_df = pd.read_csv("test.csv")

In [6]:
# Shuffle training df
train_df =train_df.sample(frac=1, random_state=42)
train_df.head()

,id,keyword,location,text,target
2644,3796,destruction,NaN,So you have a new weapon that can cause un-ima...,1
2227,3185,deluge,NaN,The f$&amp;@ing things I do for #GISHWHES Just...,0
5448,7769,police,UK,DT @georgegalloway: RT @Galloway4Mayor: ÛÏThe...,1
132,191,aftershock,NaN,Aftershock back to school kick off was great. ...,0
6845,9810,trauma,"Montgomery County, MD",in response to trauma Children of Addicts deve...,0


In [7]:
from sklearn.model_selection import train_test_split

In [8]:
train_sentences, val_sentences, train_labels, val_labels = train_test_split(train_df["text"].to_numpy(),
                                                                            train_df["target"].to_numpy(),
                                                                            test_size=0.1, # 10% of the train_df for validation_split
                                                                            random_state=42)

In [9]:
import tensorflow as tf
from keras.layers import TextVectorization
tf.__version__

'2.15.0'

In [10]:
  # use default TextVectorization parameters
  text_vectorizer = TextVectorization(
      max_tokens=10000, # How many words are in vocab
      standardize="lower_and_strip_punctuation",
      split="whitespace",
      ngrams=None, # Create groups of n-words
      output_mode="int", # How to map words to num
      output_sequence_length=50, # how long does sequences will be
      pad_to_max_tokens=True
  )

In [11]:
# Setup text vectorization variables
max_vocab_length = 10000 # max num of words to have in our vocab
max_length = 15

text_vectorizer = TextVectorization(
    max_vocab_length,
    output_mode="int", # How to map words to num
  output_sequence_length=max_length, # how long does sequences will be
)

In [12]:
 text_vectorizer.adapt(train_sentences)

In [13]:
words_in_vocab = text_vectorizer.get_vocabulary()

In [14]:
from keras import layers

embedding = layers.Embedding(
    input_dim=max_vocab_length,  # set input shape
    output_dim=16 * 8, # 128 = 16 * 8
    input_length=max_length

)

In [15]:
SAVE_DIR = "model_logs"

In [16]:
from sklearn import metrics

def evaluate_model(y_true, y_pred):
  "Evaluate a model on Accuracy, Precision, Recall and F1-score"

  accuracy_score = metrics.accuracy_score(y_true, y_pred)
  precision, recall, f1_score, _ = metrics.precision_recall_fscore_support(y_true, y_pred, average="weighted")

  return {"accuracy_score": accuracy_score,
          "precision_score": precision,
          "recall_score": recall,
           "f1_score": f1_score}

In [17]:
import tensorflow_hub as hub
embed = hub.load("https://www.kaggle.com/models/google/universal-sentence-encoder/TensorFlow2/universal-sentence-encoder/2")

In [18]:
embeddings = embed([
    "The quick brown fox jumps over the lazy dog.",
    "I am a sentence for which I would like to get its embedding"])

In [19]:
embeddings[0]

<tf.Tensor: shape=(512,), dtype=float32, numpy=
array([-3.13301720e-02, -6.33863360e-02, -1.60750095e-02, -1.03489580e-02,
       -4.65009660e-02,  3.72315794e-02,  5.91584295e-03,  7.17439801e-02,
        1.66644566e-02,  6.09076284e-02,  6.65525720e-02,  2.37051118e-02,
        5.76492981e-04,  5.68432137e-02,  2.41616610e-02, -5.33624412e-03,
        4.70477082e-02,  1.92157291e-02,  7.68255442e-02,  5.66960173e-03,
       -7.52822235e-02, -1.71372592e-02, -7.50271231e-02,  7.63734579e-02,
       -5.43796234e-02, -1.38909009e-03, -1.83018781e-02, -4.67204079e-02,
       -4.72413860e-02,  2.70678271e-02,  3.23333964e-02,  5.53706400e-02,
        3.37095521e-02, -1.37066245e-02,  5.52706048e-03, -8.22692513e-02,
        1.41951004e-02,  6.82791546e-02,  1.83204971e-02, -2.14787275e-02,
        4.14966792e-02, -2.02740170e-02, -6.01058826e-03,  2.44824439e-02,
       -8.84008929e-02, -2.56653950e-02, -3.83261442e-02, -5.61062545e-02,
        4.68127988e-02,  3.20312679e-02,  7.72727579

In [20]:
# Create a Keras Layer using the USE pretrained layer
sentence_encoder_layer = hub.KerasLayer(
    "https://www.kaggle.com/models/google/universal-sentence-encoder/TensorFlow2/universal-sentence-encoder/2",
    input_shape=[],
    dtype=tf.string,
    trainable=False,
    name="USE",
)

In [27]:
# Create model
from tensorflow.keras import layers

model_6 = tf.keras.Sequential([
    sentence_encoder_layer,
    layers.Dense(64, activation="relu"),
    layers.Dense(1, activation="sigmoid")
], name="model_6_USE")

In [28]:
model_6.summary()

Model: "model_6_USE"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 USE (KerasLayer)            (None, 512)               256797824 
                                                                 
 dense_2 (Dense)             (None, 64)                32832     
                                                                 
 dense_3 (Dense)             (None, 1)                 65        
                                                                 
Total params: 256830721 (979.73 MB)
Trainable params: 32897 (128.50 KB)
Non-trainable params: 256797824 (979.61 MB)
_________________________________________________________________


In [29]:
model_6.compile(
    optimizer=tf.keras.optimizers.Adam(),
    loss=tf.losses.binary_crossentropy,
    metrics=["accuracy"]
)

In [30]:
# Train a classifier on top of USE pretrained embeddings
model_6_history= model_6.fit(
    train_sentences,
    train_labels,
    epochs=5,
    validation_data=(val_sentences, val_labels),
    callbacks=[create_tensorboard_callback(SAVE_DIR, "tf_hub_USA")]
)

Saving TensorBoard log files to: model_logs/tf_hub_USA/20240516-080549
Epoch 1/5
215/215 [==============================] - 10s 20ms/step - loss: 0.5101 - accuracy: 0.7778 - val_loss: 0.4492 - val_accuracy: 0.7992
Epoch 2/5
215/215 [==============================] - 3s 15ms/step - loss: 0.4154 - accuracy: 0.8164 - val_loss: 0.4366 - val_accuracy: 0.8071
Epoch 3/5
215/215 [==============================] - 3s 13ms/step - loss: 0.4005 - accuracy: 0.8227 - val_loss: 0.4341 - val_accuracy: 0.8110
Epoch 4/5
215/215 [==============================] - 3s 12ms/step - loss: 0.3921 - accuracy: 0.8262 - val_loss: 0.4325 - val_accuracy: 0.8071
Epoch 5/5
215/215 [==============================] - 3s 12ms/step - loss: 0.3834 - accuracy: 0.8318 - val_loss: 0.4300 - val_accuracy: 0.8123


In [35]:
# Make preds with USE
model_6_pred_probs = model_6.predict(val_sentences)

24/24 [==============================] - 1s 27ms/step


In [36]:
model_6_preds = tf.squeeze(tf.round(model_6_pred_probs))

In [37]:
evaluate_model(val_labels, model_6_preds)

{'accuracy_score': 0.8123359580052494,
 'precision_score': 0.8163219290181186,
 'recall_score': 0.8123359580052494,
 'f1_score': 0.8102465800135692}